In [ ]:
#%pip install pandas openpyxl numpy ipykernel nbformat plotly #--------------Rodar só 1(uma) Vez!!!
#pip install matplotlib
#import pandas
#import pandas as pd

#-------------------------------------------------Filtrar Valores de Acordo com Determinada Data
import pandas as pd
#--------------Leitura d-arqv CSV
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
#---------------Solict-data d-user
dataReq = input("Data (formato: dd/mm/aaaa): ")
#---------------Filtr-table pela data fornecd
def data_Requ_valorData(dataReq):
    valoresData = dataReq
    if valoresData in tabela['Data Req.'].values:
        consulta = tabela[tabela['Data Req.'] == valoresData]
        display(consulta)
    else:
        print("Não há valores relacionados com essa Data!")
#------------Call Function c/-date fornecd
data_Requ_valorData(dataReq)

In [ ]:
#----------------------------------------------Veículos com Média Km/Litro
#-----------------------------------------------------MAIS BAIXA!!

import pandas as pd
import plotly.express as px
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
#Solicit mês/ano d'user
mes = input("Digite o mês (01-12): ")
ano = input("Digite o ano (AAAA): ")
#Coluns p/excl
colunas_para_excluir = ["Requisição", "Horim. Equip.", "Hora Abast.", "Obs.", "Abast. Externo"]
colunas_existentes = [col for col in colunas_para_excluir if col in tabela.columns]
if colunas_existentes:
    tabela = tabela.drop(columns=colunas_existentes)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")
#Tratament->coluns 'Litros'/'Km Rodados'
tabela['Litros'] = tabela['Litros'].astype(str).str.replace(',', '').str[:-2]
tabela['Km Rodados'] = pd.to_numeric(tabela['Km Rodados'], errors='coerce')
tabela['Litros'] = pd.to_numeric(tabela['Litros'], errors='coerce')
tabela = tabela.dropna(subset=['Km Rodados', 'Litros', 'Veículo/Equip.'])
#Filtr p/excl o'GERADOR'
tabela = tabela[tabela['Veículo/Equip.'] != 'GERADOR']
#Filtr tbl by-mês/ano fornecds
tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], format='%d/%m/%Y', errors='coerce')
tabela = tabela.dropna(subset=['Data Req.'])
tabela_filtrada = tabela[(tabela['Data Req.'].dt.month == int(mes)) & (tabela['Data Req.'].dt.year == int(ano))]
#Calc'Km por Litro'/arredond p/3casas decims
tabela_filtrada['Km por Litro'] = (tabela_filtrada['Km Rodados'] / tabela_filtrada['Litros']).round(3)
#Calc méd d'Km por Litro' p/ cad'Veículo/Equip.'/arredond p/3casas decims
media_km_por_litro_por_veiculo = tabela_filtrada.groupby('Veículo/Equip.')['Km por Litro'].mean().round(3).reset_index()
#Renome-colun p/ diferenciaç
media_km_por_litro_por_veiculo = media_km_por_litro_por_veiculo.rename(columns={'Km por Litro': 'Media Km por Litro'})
#Mescl méds dvolt t/tbl orignl
tabela_filtrada = tabela_filtrada.merge(media_km_por_litro_por_veiculo, on='Veículo/Equip.', how='left')
#Filtr veícls c/méd d'Km por Litro'+ baixa
veiculos_abaixo_da_media = media_km_por_litro_por_veiculo.nsmallest(25, 'Media Km por Litro')['Veículo/Equip.']
#Filtr tbl p/ includ only those veícls
tabela_filtrada = tabela_filtrada[tabela_filtrada['Veículo/Equip.'].isin(veiculos_abaixo_da_media)]
#Gráfico comparand
grafico = px.bar(tabela_filtrada, x='Veículo/Equip.', y='Km por Litro', color='Media Km por Litro',#.histogram
                       title="Comparação dos Veículos/Equipamentos com Média de Km por Litro Mais Baixa",
                       hover_data=['Requisitante'])

grafico.update_layout(xaxis_title="Veículo/Equip.",
                      yaxis_title="Km por Litro",
                      xaxis_tickangle=-45,
                      bargap=0.2)  #Ajust-espaçament entr barras
grafico.show()

In [ ]:
#----------------------------------------------------------Veículos/Equip com MENOR lucro(Km Rodados - Vlr. Total)
#-----------------------------------------------------------------------Junto do Requisitante!!

import pandas as pd
import plotly.express as px
#Read arqv CSV
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
#Solicit mês/ano d'user
mes = input("Digite o mês (01-12): ")
ano = input("Digite o ano (AAAA): ")
#Coluns p/ excl
colunas_para_excluir = ["Requisição", "Hora Abast.", "Obs.", "Abast. Externo", "Combustível", "Horim. Equip.", "Vlr. Unitário"]
colunas_existentes = [col for col in colunas_para_excluir if col in tabela.columns]
if colunas_existentes:
    tabela = tabela.drop(columns=colunas_existentes)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")
#Remoç d'NaNs
tabela = tabela.dropna()
#Tratamnt coluns'Litros'/'Km Rodados'
tabela['Litros'] = tabela['Litros'].astype(str).str.replace(',', '').str[:-2]
tabela['Vlr. Total'] = tabela['Vlr. Total'].astype(str).str.replace(',', '')#.str[:-2]
tabela['Km Rodados'] = pd.to_numeric(tabela['Km Rodados'], errors='coerce')
tabela['Litros'] = pd.to_numeric(tabela['Litros'], errors='coerce')
tabela['Vlr. Total'] = pd.to_numeric(tabela['Vlr. Total'], errors='coerce')
tabela['Km Atual'] = pd.to_numeric(tabela['Km Atual'], errors='coerce')
#print(tabela.dtypes)
tabela = tabela.dropna(subset=['Km Rodados', 'Litros', 'Vlr. Total', 'Km Atual', 'Veículo/Equip.', 'Requisitante'])
#Filtr tbl pelo mês/ano fornecds
tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], format='%d/%m/%Y', errors='coerce')
tabela = tabela.dropna(subset=['Data Req.'])
tabela_filtrada = tabela[(tabela['Data Req.'].dt.month == int(mes)) & (tabela['Data Req.'].dt.year == int(ano))]
#Agrup dados por'Veículo/Equip.'/'Requisitante'
agrupado = tabela_filtrada.groupby(['Veículo/Equip.', 'Requisitante']).agg({
    'Litros': 'sum',
    'Vlr. Total': 'sum',
    'Km Rodados': 'sum',
    'Km Atual': 'max'
}).reset_index()
#Calc 'Lucro'/arredond p/4 casas decims
agrupado['Lucro'] = (agrupado['Km Rodados'] - agrupado['Vlr. Total']).round(4)
#Select-20 veícls c/less lucr
bottom20_lucro = agrupado.nsmallest(20, 'Lucro')
#Criaç gráfc
grafico = px.bar(bottom20_lucro, x='Veículo/Equip.', y='Lucro', color='Lucro',
                 color_continuous_scale='Viridis',
                 hover_data=['Requisitante'])
grafico.update_layout(title="Bottom 20 Veículos/Equipamentos por Lucro",
                      xaxis_title="Veículo/Equip.",
                      yaxis_title="Lucro (Km Rodados - Vlr. Total)",
                      xaxis_tickangle=-45,
                      bargap=0.2)  #Ajust d'espaçament entr barras
grafico.show()